# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name) ------didn't work
#conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')

#conn = pyodbc.connect("DSN=COLLEGE;"
#                      "Server=DESKTOP-4P3K6TB\SQLEXPRESS01;" ------didn't work
#                      "Database=COLLEGE;"
#                      "Trusted_Connection=yes;")

conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-4P3K6TB\SQLEXPRESS01;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")

#conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};" -----for class
#                      "Server=192.168.1.1;"
#                      "Database=COLLEGE;"
#                      "uid=dsuser06;"
#                      "pwd=DSuser06!;"
#                     )


In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 

In [4]:
### Get the classrooms table
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 

In [5]:
### Get the courses table
courses = pd.read_sql("SELECT * FROM Courses;",conn) 

In [6]:
### Get the departments table
departments = pd.read_sql("SELECT * FROM Departments;",conn) 

In [7]:
### Get the teachers table
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 

In [8]:
co_cl_dep = pd.merge(pd.merge(courses, classrooms, on='CourseId', how='inner'), departments, left_on='DepartmentID', right_on='DepartmentId')

# Questions
 
 
## Q1. Count the number of students on each department

In [9]:
co_cl_dep.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName
0,1,English Begginers,1,15,202,92.0,1,English
1,1,English Begginers,1,15,12,81.0,1,English
2,1,English Begginers,1,15,232,82.0,1,English
3,1,English Begginers,1,15,123,62.0,1,English
4,1,English Begginers,1,15,274,73.0,1,English


In [10]:
dep_count = co_cl_dep.loc[:, ['DepartmentName','StudentId']].drop_duplicates().groupby('DepartmentName').agg({'StudentId':['count']})

In [11]:
dep_count

,StudentId
,count
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [12]:
co_count_eng = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 1),
              ['CourseName','StudentId']].drop_duplicates().groupby('CourseName').agg({'StudentId':['count']})


In [13]:
co_count_eng

,StudentId
,count
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38


In [14]:
co_count_eng.loc['Total'] = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 1),
              'StudentId'].drop_duplicates().agg({'StudentId':['count']})

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  


In [15]:
co_count_eng

,StudentId
,count
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38
Total,84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [16]:
co_count_sci = co_cl_dep.loc[(co_cl_dep['DepartmentID'] == 2),
              ['CourseName','StudentId']].drop_duplicates().groupby('CourseName').agg({'StudentId':['count']})

In [17]:
co_count_sci.loc[(co_count_sci['StudentId']['count'] < 22), 'ClassroomSize'] = 'Small'
co_count_sci.loc[(co_count_sci['StudentId']['count'] >= 22), 'ClassroomSize'] = 'Big'

In [18]:
co_count_sci

,StudentId,ClassroomSize
,count,
CourseName,,
Algebra,12,Small
Biology,36,Big
Chemistry,23,Big
Computer Science,38,Big
Data Science,30,Big
Genetics,37,Big
Geometry,13,Small
Java,16,Small


In [19]:
co_count_sci.groupby('ClassroomSize').agg({'ClassroomSize':['count']})

,ClassroomSize
,count
ClassroomSize,
Big,10
Small,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [20]:
students.drop_duplicates().groupby('Gender').agg({'StudentId':['count']})

,StudentId
,count
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [21]:
co_cl_dep_st = pd.merge(co_cl_dep, students, on='StudentId', how='inner')
co_cl_dep_st.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName,FirstName,LastName,Gender
0,1,English Begginers,1,15,202,92.0,1,English,Autumn,Major,F
1,4,Mathematics 101,2,2,202,73.0,2,Science,Autumn,Major,F
2,8,Chemistry,2,14,202,84.0,2,Science,Autumn,Major,F
3,18,R,2,17,202,66.0,2,Science,Autumn,Major,F
4,20,History of Arts,3,8,202,64.0,3,Arts,Autumn,Major,F


In [22]:
gen_co = co_cl_dep_st.loc[:,['CourseName','StudentId','Gender']].drop_duplicates().\
groupby(['CourseName','Gender']).agg({'StudentId':['count']})
gen_co.reset_index(inplace=True)
gen_co.head()

,CourseName,Gender,StudentId
,,,count
0,Advanced English,F,13
1,Advanced English,M,11
2,Algebra,F,8
3,Algebra,M,4
4,Biology,F,19


In [23]:
gen_co_t = co_cl_dep_st.loc[:,['CourseName','StudentId','Gender']].drop_duplicates().\
groupby(['CourseName']).agg({'StudentId':['count']})
gen_co_t.reset_index(inplace=True)
gen_co_t.head()

,CourseName,StudentId
,,count
0,Advanced English,24
1,Algebra,12
2,Biology,36
3,Chemistry,23
4,Computer Science,38


In [24]:
gen_co_m = pd.merge(gen_co_t, gen_co, on='CourseName', how='left')
gen_co_m['StudentsPercent'] = gen_co_m['StudentId_y']['count'] / gen_co_m['StudentId_x']['count'] * 100
gen_co_m.head()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


,CourseName,StudentId_x,Gender,StudentId_y,StudentsPercent
,,count,,count,
0,Advanced English,24,F,13,54.166667
1,Advanced English,24,M,11,45.833333
2,Algebra,12,F,8,66.666667
3,Algebra,12,M,4,33.333333
4,Biology,36,F,19,52.777778


In [25]:
gen_co_f = gen_co_m.loc[gen_co_m['StudentsPercent']>70,['CourseName','Gender','StudentsPercent']]
gen_co_f

,CourseName,Gender,StudentsPercent
,,,
14,Genetics,F,70.270270
46,Sculpture,F,73.913043
52,Tenis,F,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [26]:
dep_st_av = co_cl_dep_st.loc[:,['DepartmentName','StudentId','Degree']].groupby(['DepartmentName','StudentId']).agg({'Degree':['mean']})
dep_st_av.head()

Degree
                           mean
DepartmentName StudentId       
Arts           1           87.0
               2           79.5
               4           69.0
               5           64.0
               6           69.0

In [27]:
dep_st_high_deg = dep_st_av[dep_st_av['Degree']['mean'] > 80]

In [28]:
dep_st_high_deg.reset_index(inplace=True)
dep_st_high_deg.groupby(['DepartmentName']).agg({'StudentId':['count']})

,StudentId
,count
DepartmentName,
Arts,47
English,36
Science,84
Sport,32


## Q7. For each department, how many students passed with a grades under 60?

In [32]:
dep_st_low_deg = dep_st_av[dep_st_av['Degree']['mean'] < 60]
dep_st_low_deg.reset_index(inplace=True)

In [33]:
dep_st_low_deg.groupby(['DepartmentName']).agg({'StudentId':['count']})

,StudentId
,count
DepartmentName,
Arts,3
English,1
Science,2
Sport,1


## Q8. Rate the teachers by their average student's grades (in descending order).

In [47]:
co_cl_dep_st_te = pd.merge(co_cl_dep_st,teachers, on='TeacherId', how='left', suffixes=('_St', '_Te')).drop_duplicates()
co_cl_dep_st_te.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName,FirstName_St,LastName_St,Gender_St,FirstName_Te,LastName_Te,Gender_Te
0,1,English Begginers,1,15,202,92.0,1,English,Autumn,Major,F,Conor,Mckittrick,M
1,4,Mathematics 101,2,2,202,73.0,2,Science,Autumn,Major,F,Amanda,Dworkin,F
2,8,Chemistry,2,14,202,84.0,2,Science,Autumn,Major,F,Jeffrey,Batzel,M
3,18,R,2,17,202,66.0,2,Science,Autumn,Major,F,Tanner,Fitzsimons,M
4,20,History of Arts,3,8,202,64.0,3,Arts,Autumn,Major,F,Caroline,Knuesel,F


In [48]:
co_cl_dep_st_teacher = co_cl_dep_st_te.copy()
co_cl_dep_st_teacher['TeacherName'] = co_cl_dep_st_te['FirstName_Te'] + ' ' + co_cl_dep_st_te['LastName_Te']
co_cl_dep_st_teacher.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName,FirstName_St,LastName_St,Gender_St,FirstName_Te,LastName_Te,Gender_Te,TeacherName
0,1,English Begginers,1,15,202,92.0,1,English,Autumn,Major,F,Conor,Mckittrick,M,Conor Mckittrick
1,4,Mathematics 101,2,2,202,73.0,2,Science,Autumn,Major,F,Amanda,Dworkin,F,Amanda Dworkin
2,8,Chemistry,2,14,202,84.0,2,Science,Autumn,Major,F,Jeffrey,Batzel,M,Jeffrey Batzel
3,18,R,2,17,202,66.0,2,Science,Autumn,Major,F,Tanner,Fitzsimons,M,Tanner Fitzsimons
4,20,History of Arts,3,8,202,64.0,3,Arts,Autumn,Major,F,Caroline,Knuesel,F,Caroline Knuesel


In [49]:
te_st_av = co_cl_dep_st_teacher.loc[:,['TeacherName','StudentId','Degree']].groupby(['TeacherName']).agg({'Degree':['mean']})
te_st_av.reset_index(inplace=True)
te_st_av.sort_values(by=('Degree', 'mean'), ascending=False)

,TeacherName,Degree
,,mean
5,Darick Hess,81.187500
9,Jordyn Baron,80.722222
7,Heather Metcalf,80.500000
12,Kaylea Monzingo,79.342105
21,Zachary Healy,79.333333
4,Daniel Bland,78.950000
13,Keegan Price,78.777778
16,Maxwell Romkee,78.708333
3,Conor Mckittrick,78.702128


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [50]:
college = co_cl_dep_st_te.loc[:,['CourseId','CourseName','DepartmentName','FirstName_Te', 'LastName_Te']].drop_duplicates()
college.head()

,CourseId,CourseName,DepartmentName,FirstName_Te,LastName_Te
0,1,English Begginers,English,Conor,Mckittrick
1,4,Mathematics 101,Science,Amanda,Dworkin
2,8,Chemistry,Science,Jeffrey,Batzel
3,18,R,Science,Tanner,Fitzsimons
4,20,History of Arts,Arts,Caroline,Knuesel


In [51]:
co_cl_dep_st.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName,FirstName,LastName,Gender
0,1,English Begginers,1,15,202,92.0,1,English,Autumn,Major,F
1,4,Mathematics 101,2,2,202,73.0,2,Science,Autumn,Major,F
2,8,Chemistry,2,14,202,84.0,2,Science,Autumn,Major,F
3,18,R,2,17,202,66.0,2,Science,Autumn,Major,F
4,20,History of Arts,3,8,202,64.0,3,Arts,Autumn,Major,F


In [52]:
co_st_cnt = co_cl_dep_st.loc[:,['CourseId','StudentId']].drop_duplicates().dropna().groupby(['CourseId']).agg({'StudentId':['count']})

In [53]:
co_st_cnt.head()

,StudentId
,count
CourseId,
1,32
2,24
3,38
4,18
5,10


In [55]:
college_sum = pd.merge(college,co_st_cnt, on='CourseId', how='left')
college_sum.head()

,CourseId,CourseName,DepartmentName,FirstName_Te,LastName_Te,"(StudentId, count)"
0,1,English Begginers,English,Conor,Mckittrick,32
1,4,Mathematics 101,Science,Amanda,Dworkin,18
2,8,Chemistry,Science,Jeffrey,Batzel,23
3,18,R,Science,Tanner,Fitzsimons,33
4,20,History of Arts,Arts,Caroline,Knuesel,32


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [56]:
st_co_cl_dep = pd.merge(co_cl_dep, students, on="StudentId", how="left").drop_duplicates()

In [57]:
st_cl_num = st_co_cl_dep.loc[:, ['StudentId','CourseId']].drop_duplicates().groupby(['StudentId']).agg({'CourseId':['count']})

In [58]:
st_cl_num.head()

,CourseId
,count
StudentId,
1,4
2,5
3,3
4,3
5,3


In [59]:
st_cl_num.reset_index(inplace=True)

In [60]:
st_cl_av = st_co_cl_dep.loc[:, ['StudentId','CourseId','Degree']].drop_duplicates().groupby(['StudentId','CourseId']).agg({'Degree':['mean']})

In [61]:
st_cl_av.head()

Degree
                     mean
StudentId CourseId       
1         10         86.0
          11         79.0
          15         73.0
          22         87.0
2         11         73.0

In [62]:
st_cl_av_piv = st_cl_av.pivot_table(
        values='Degree',
        index='StudentId', 
        columns='CourseId')

In [66]:
st_cl_av_piv

mean                                                         ...  \
CourseId     1     2     3     4     5     6     7     8     9      10  ...   
StudentId                                                               ...   
1           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   86.0  ...   
2           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
3          84.0   NaN   NaN   NaN   NaN  76.0   NaN   NaN   NaN    NaN  ...   
4          71.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
5           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  100.0  ...   
6           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
7           NaN   NaN   NaN   NaN   NaN   NaN   NaN  83.0   NaN    NaN  ...   
8           NaN   NaN  89.0   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
9           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   88.0  ...   
10          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
11          NaN   NaN   NaN   NaN   NaN  76.0   NaN   NaN   NaN    NaN  ...   
12         81.0  85.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
13          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   65.0  ...   
15          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  80.0    NaN  ...   
16          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
17          NaN   NaN   NaN  83.0   NaN   NaN   NaN   NaN   NaN    NaN  ...   
18         83.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
19          NaN   NaN   NaN   NaN   NaN   NaN   NaN  78.0   NaN    NaN  ...   
20          NaN   NaN   NaN   NaN  81.0   NaN   NaN   NaN   NaN    NaN  ...   
21          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
22         90.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
23          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
24          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
25          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
26          NaN  96.0  83.0   NaN   NaN   NaN  91.0   NaN   NaN    NaN  ...   
27          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
28          NaN   NaN  86.0   NaN   NaN  63.0  76.5   NaN   NaN    NaN  ...   
29          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
30         76.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  81.0    NaN  ...   
31          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
...         ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...   
247         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
248         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
249         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
250         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
251        81.0   NaN  80.0   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
252         NaN   NaN   NaN  84.0   NaN   NaN   NaN   NaN   NaN    NaN  ...   
253        94.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  69.0    NaN  ...   
254         NaN   NaN   NaN  76.0   NaN   NaN   NaN   NaN   NaN    NaN  ...   
255         NaN   NaN   NaN   NaN   NaN   NaN   NaN  74.0   NaN    NaN  ...   
257         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   86.0  ...   
258         NaN   NaN  84.0   NaN   NaN  84.0   NaN   NaN   NaN   97.0  ...   
259         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
260         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   77.0  ...   
262         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
263         NaN   NaN  76.0   NaN   NaN   NaN   NaN  62.0   NaN    NaN  ...   
265         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  ...   
266         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   N

In [67]:
st_av = st_co_cl_dep.loc[:, ['StudentId','Degree']].drop_duplicates().groupby(['StudentId']).agg({'Degree':['mean']})

In [68]:
st_av.head()

,Degree
,mean
StudentId,
1,81.250000
2,78.500000
3,80.000000
4,71.333333
5,76.666667


In [69]:
st_av.reset_index(inplace=True)

In [70]:
st_av.head()

,StudentId,Degree
,,mean
0,1,81.250000
1,2,78.500000
2,3,80.000000
3,4,71.333333
4,5,76.666667


In [71]:
student_1 = pd.merge(co_cl_dep_st.loc[:, ['StudentId','FirstName','LastName']],st_cl_num, on='StudentId',how='left').drop_duplicates()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [72]:
student_1.rename(columns={('CourseId','count'): 'NumOfCourses'}, inplace=True)

In [73]:
student_1.head()

,StudentId,FirstName,LastName,NumOfCourses
0,202,Autumn,Major,7
7,12,Sophiya,Aponte,4
11,232,Victoria,Kishinevsky,3
14,123,Kelsi,Taber,2
16,274,Samantha,Anderson,1


In [74]:
co_cl_dep_st.head()

,CourseId,CourseName,DepartmentID,TeacherId,StudentId,Degree,DepartmentId,DepartmentName,FirstName,LastName,Gender
0,1,English Begginers,1,15,202,92.0,1,English,Autumn,Major,F
1,4,Mathematics 101,2,2,202,73.0,2,Science,Autumn,Major,F
2,8,Chemistry,2,14,202,84.0,2,Science,Autumn,Major,F
3,18,R,2,17,202,66.0,2,Science,Autumn,Major,F
4,20,History of Arts,3,8,202,64.0,3,Arts,Autumn,Major,F


In [75]:
student_2 = pd.merge(student_1,st_av,on='StudentId',how='left').drop_duplicates()

In [76]:
st_cl_av_piv.reset_index(inplace=True)

In [77]:
student_2.rename(columns={('Degree','mean'): 'MeanDegree'}, inplace=True)
students_sum = pd.merge(student_2,st_cl_av_piv,on='StudentId', how='left')
students_sum

,StudentId,FirstName,LastName,NumOfCourses,MeanDegree,"(mean, 1)","(mean, 2)","(mean, 3)","(mean, 4)","(mean, 5)",...,"(mean, 20)","(mean, 21)","(mean, 22)","(mean, 23)","(mean, 24)","(mean, 25)","(mean, 27)","(mean, 29)","(mean, 30)","(mean, 31)"
0,202,Autumn,Major,7,75.800000,92.0,NaN,NaN,73.0,NaN,...,64.0,NaN,NaN,NaN,NaN,73.0,NaN,73.0,NaN,NaN
1,12,Sophiya,Aponte,4,78.500000,81.0,85.0,NaN,NaN,NaN,...,NaN,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN
2,232,Victoria,Kishinevsky,3,82.000000,82.0,NaN,NaN,NaN,NaN,...,NaN,NaN,79.0,NaN,NaN,NaN,85.0,NaN,NaN,NaN
3,123,Kelsi,Taber,2,64.000000,62.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,274,Samantha,Anderson,1,73.000000,73.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,150,Emily,Beach,4,87.250000,80.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,98.0,NaN,86.0,NaN,NaN,NaN,NaN
6,161,Cullen,Brewton,7,75.285714,65.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,76.0,82.0,NaN,NaN,91.0,NaN
7,39,Kayli,May,2,91.000000,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,251,Heather,Williams,4,81.250000,81.0,NaN,80.0,NaN,NaN,...,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN
9,30,Kyle,Nibbelink,3,84.666667,76.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
